In [101]:
%matplotlib inline

import numpy as np
import keras
import matplotlib.pyplot as plt
import pysam
import argparse
import h5py
        
from keras.datasets import mnist, cifar10
from keras.layers import Input, Dense
from keras.models import Model

In [26]:
cram = "/dsde/data/datasets/SnapShotExperiment2015/CEUTrio/Alignments/G94982.NA12878/NA12878.cram"
samfile = pysam.AlignmentFile(cram, "rc")	# "rb" for bam, "rc" for cram

In [107]:
# for read in samfile.fetch(args.chrom, args.start_pos, args.end_pos):
fetcher = samfile.fetch("chr1")

p_sampling = 0.5
num_tr_examples = int(1e6)
num_test_examples = int(1e5)
read_length = 151

X_tr = np.zeros((num_tr_examples, read_length))
X_test = np.zeros((num_test_examples, read_length))
i=0
j=0
for read in samfile.fetch("chr1"):
    if i == num_tr_examples + num_test_examples:
        break;
    
    # QC the reads
    read_group = read.get_tag('RG')
    if 'artificial' in read_group.lower():
        continue
    if not read.is_proper_pair or not read.is_paired:
        continue
    if read.is_duplicate or read.is_secondary or read.is_supplementary or read.is_qcfail or read.is_unmapped:
        continue

    # randomly sample from among the good reads
    dice = np.random.rand()
    if dice > p_sampling:
        continue    
        
    bqs = np.array(list(reversed(read.query_qualities))) if read.is_reverse else np.array(read.query_qualities)
    if (i < num_tr_examples):
        X_tr[i] = bqs
        i += 1
        continue
    else: 
        X_test[j] = bqs
        j += 1
        i += 1

train_file_name="/dsde/data/deep/takutoencoder/x_tr.npy"
test_file_name="/dsde/data/deep/takutoencoder/x_test.npy"
np.save(train_file_name, X_tr)
np.save(test_file_name, X_test)

In [99]:
print int(1e8)
print X[num_examples-1]
print X.shape

100000000
[30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 40. 30. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 30. 30. 30. 30. 30. 40. 30. 30. 30. 30. 30. 30. 30.
 30. 30. 30. 30. 30. 20. 30.]
(1000000, 151)


In [108]:
np.save(test_file_name, X_test)

In [100]:
file_name="/dsde/data/deep/takutoencoder/x.npy"
np.save(file_name, X)

In [103]:
# take the sample code from https://blog.keras.io/building-autoencoders-in-keras.html

# this is the size of our encoded representations
encoding_dim = 32

input_qualities = Input(shape=(read_length,))
encoded = Dense(encoding_dim, activation='relu')(input_qualities)
decoded = Dense(read_length, activation='linear')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_qualities, decoded)

# separate encoder model
encoder = Model(input_img, encoded)

In [105]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [110]:
autoencoder.fit(X_tr, X_tr,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 1000000 samples, validate on 100000 samples
Epoch 1/10
1000000/1000000 [==============================] - 11s 11us/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/10
 242176/1000000 [======>.......................] - ETA: 8s - loss: 0.0000e+00

KeyboardInterrupt: 